<h1>Affordability</h1>

Measuring affordability of postcode based on median income, median mortage, median rental price and average facilities

In [1]:
# importing necessary libraries
import pandas as pd

In [5]:
# Reading in median rental price per postcode
median_rent = pd.read_csv("../data/curated/median_rental_postcode.csv")
#median_rent.head()

,Postcode,Cost
0,3000,510.0
1,3002,642.5
2,3003,520.0
3,3004,550.0
4,3006,551.5


In [9]:
# Reading in preprocessed property data
property_data = pd.read_csv("../data/curated/properties_processed.csv")
# Remove unecessary column "Unnamed: 0"
property_data = property_data.drop(columns = ["Unnamed: 0"])
property_data.head()

,Name,Cost,Coordinates,Bed,Bath,Parking,Property_Type,Agency,Postcode
0,5408/500 Elizabeth Street Melbourne VIC 3000,440.0,"[-37.8072443, 144.9602814]",1,1,0,Apartment / Unit / Flat,BRADY residential,3000
1,502/118 Russell Street Melbourne VIC 3000,620.0,"[-37.8135864, 144.9687232]",1,1,0,Apartment / Unit / Flat,Dingle Partners,3000
2,202A/441 Lonsdale Street Melbourne VIC 3000,300.0,"[-37.8134292, 144.9594445]",1,1,0,Apartment / Unit / Flat,Biggin & Scott Stonnington,3000
3,57/243 Collins Street Melbourne VIC 3000,400.0,"[-37.8159969, 144.9657956]",1,1,0,Apartment / Unit / Flat,Harcourts Melbourne City,3000
4,2311/601 Little Lonsdale Street Melbourne VIC ...,625.0,"[-37.8137564, 144.9537143]",2,2,1,Apartment / Unit / Flat,Harcourts Melbourne City,3000


In [15]:
# Calculate the average number of facilities per postcode
avg_bed = property_data.groupby(by = "Postcode")["Bed"].mean()
#avg_bed.head()
avg_bath = property_data.groupby(by = "Postcode")["Bath"].mean()
#avg_bath.head()
avg_parking = property_data.groupby(by = "Postcode")["Parking"].mean()
#avg_parking.head()

Postcode
3000    0.365517
3002    1.000000
3003    0.609375
3004    1.096774
3006    0.647959
Name: Parking, dtype: float64

In [30]:
# Create dataframe of average facilities per postcode
avg_facilities = pd.DataFrame()
avg_facilities["Postcode"] = property_data["Postcode"].unique()
avg_facilities["Average # Beds"] = avg_bed.to_list()
avg_facilities["Average # Baths"] = avg_bath.to_list()
avg_facilities["Average # Parking"] = avg_parking.to_list()
#avg_facilities.head()

,Postcode,Average # Beds,Average # Baths,Average # Parking
0,3000,1.579310,1.296552,0.365517
1,3002,1.909091,1.318182,1.000000
2,3003,1.796875,1.343750,0.609375
3,3004,1.763441,1.494624,1.096774
4,3006,1.785714,1.469388,0.647959
